In [1]:
# Cargar librerias

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Cargar datos
df_orignal = pd.read_csv('202220_Laboratorio_2_data_Empresas_mas_grandes_del_pais_2018.csv', sep=';', encoding = 'utf-8', index_col=0, decimal=',')

# Entendimiento y preparación de los datos

In [3]:
df_orignal.shape

(1068, 19)

In [4]:
df_orignal.head()

,NIT,RAZON SOCIAL,SUPERVISOR,REGIÓN,DEPARTAMENTO DOMICILIO,CIUDAD DOMICILIO,CIIU,MACROSECTOR,INGRESOS OPERACIONALES\n2018*,GANANCIA (PERDIDA) 2018,TOTAL ACTIVOS 2018,TOTAL PASIVOS 2018,TOTAL PATRIMONIO 2018,INGRESOS OPERACIONALES\n2017*,GANANCIA (PERDIDA) 2017,TOTAL ACTIVOS 2017,TOTAL PASIVOS 2017,TOTAL PATRIMONIO 2017,GRUPO EN NIIF
No.,,,,,,,,,,,,,,,,,,,
1,860003563,ABB LTDA,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA-D.C.-BOGOTA D.C.,"C2711 - Fabricación de motores, generadores y ...",MANUFACTURA,3.571775e+09,-5.872548e+07,2.956848e+09,2.476739e+09,4.801089e+08,4.005049e+09,-8.752900e+07,2.875460e+09,$ 2341172480,5.342871e+08,NIIF PLENAS-GRUPO 1
2,900138670,C.I BIOCOSTA S.A,SUPERSOCIEDADES,Costa Atlántica,MAGDALENA,SANTA-MARTA-MAGDALENA,G4610 - Comercio al por mayor a cambio de una ...,COMERCIO,6.018902e+09,3.504944e+07,1.207011e+09,1.031424e+09,1.755873e+08,4.994552e+09,1.083777e+07,1.020832e+09,$ 894986480,1.258455e+08,NIIF PLENAS-GRUPO 1
3,890302629,CONSTRUCTORA MELENDEZ S.A.,SUPERSOCIEDADES,Costa Pacífica,VALLE,CALI-VALLE,F4111 - Construcción de edificios residenciales,CONSTRUCCIÓN,1.739284e+09,1.735105e+07,3.030767e+09,2.163544e+09,8.672231e+08,1.039942e+09,-3.877247e+07,2.615650e+09,$ 1748991200,8.666590e+08,NIIF PLENAS-GRUPO 1
4,811004055,EMDISALUD,SUPERSALUD,Costa Atlántica,CORDOBA,MONTERIA,O8430- Actividades de planes de seguridad soci...,SERVICIOS,4.033887e+10,-3.016582e+10,9.908133e+09,4.950861e+10,-3.960047e+10,3.696528e+10,-1.086227e+10,9.807867e+09,$ 19246633716,-9.438767e+09,NIIF PYMES-GRUPO 2
5,890503314,CERAMICA ITALIA SA,SUPERSOCIEDADES,Centro - Oriente,NORTE DE SANTANDER,CUCUTA-NORTE DE SANTANDER,C2392 - Fabricación de materiales de arcilla p...,MANUFACTURA,1.419514e+09,4.582130e+06,1.498588e+09,4.971313e+08,1.001457e+09,1.360935e+09,3.250460e+07,1.509246e+09,$ 490200920,1.019045e+09,NIIF PYMES-GRUPO 2


In [5]:
# Revisar las características de los datos: completitud y tipo de dato
df_orignal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068 entries, 1 to 1068
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   NIT                           1068 non-null   int64  
 1   RAZON SOCIAL                  1068 non-null   object 
 2   SUPERVISOR                    1068 non-null   object 
 3   REGIÓN                        1067 non-null   object 
 4   DEPARTAMENTO DOMICILIO        1063 non-null   object 
 5   CIUDAD DOMICILIO              1064 non-null   object 
 6   CIIU                          1067 non-null   object 
 7   MACROSECTOR                   1065 non-null   object 
 8   INGRESOS OPERACIONALES
2018*  1067 non-null   float64
 9   GANANCIA (PERDIDA) 2018       1067 non-null   float64
 10  TOTAL ACTIVOS 2018            1066 non-null   float64
 11  TOTAL PASIVOS 2018            1066 non-null   float64
 12  TOTAL PATRIMONIO 2018         1064 non-null   float64
 13  ING

In [6]:
# Los datos parecen estar bastante completos por lo cual,
# se considera razonable eliminar los datos con valors nulos

df_prep = df_orignal.dropna()

In [15]:
# Quitar los datos duplicados

df_prep = df_prep.drop_duplicates()

In [7]:
# Convertir variables que deberian ser númericas
df_prep['TOTAL PASIVOS 2017'] = df_prep['TOTAL PASIVOS 2017'].str.extract('(\d+)', expand=False)
df_prep['TOTAL PASIVOS 2017'] = df_prep['TOTAL PASIVOS 2017'].astype('float64')
df_prep['TOTAL PASIVOS 2017']

No.
1       2.341172e+09
2       8.949865e+08
3       1.748991e+09
4       1.924663e+10
5       4.902009e+08
            ...     
1064    3.969365e+08
1065    5.435688e+08
1066    4.872296e+08
1067    6.173932e+08
1068    1.459223e+09
Name: TOTAL PASIVOS 2017, Length: 1038, dtype: float64

In [8]:
# Describit los datos
df_prep.describe()

,NIT,INGRESOS OPERACIONALES\n2018*,GANANCIA (PERDIDA) 2018,TOTAL ACTIVOS 2018,TOTAL PASIVOS 2018,TOTAL PATRIMONIO 2018,INGRESOS OPERACIONALES\n2017*,GANANCIA (PERDIDA) 2017,TOTAL ACTIVOS 2017,TOTAL PASIVOS 2017,TOTAL PATRIMONIO 2017
count,1.038000e+03,1.038000e+03,1.038000e+03,1.038000e+03,1.038000e+03,1.038000e+03,1.038000e+03,1.038000e+03,1.038000e+03,1.037000e+03,1.038000e+03
mean,8.591009e+08,1.151194e+10,6.120565e+08,1.141401e+10,6.497901e+09,6.076092e+11,9.618569e+09,1.249292e+08,1.017239e+10,5.563843e+09,4.757360e+09
std,3.780352e+07,4.371605e+10,5.109456e+09,4.685516e+10,2.663662e+10,1.941801e+13,3.571235e+10,5.265554e+09,4.350808e+10,2.536319e+10,2.296076e+10
min,8.000003e+08,1.343546e+09,-6.993593e+10,7.329964e+07,9.900000e+01,-7.303930e+10,0.000000e+00,-7.977997e+10,6.788196e+07,6.309000e+04,-6.786768e+10
25%,8.300113e+08,1.803616e+09,9.589688e+06,1.214538e+09,6.691890e+08,3.063245e+08,1.586898e+09,2.894518e+06,1.092483e+09,5.924721e+08,2.617120e+08
50%,8.600590e+08,2.801874e+09,6.791505e+07,2.517761e+09,1.380143e+09,8.597856e+08,2.502846e+09,5.014801e+07,2.207021e+09,1.187684e+09,7.468144e+08
75%,8.910601e+08,6.470835e+09,2.633236e+08,6.461248e+09,3.621377e+09,2.656821e+09,5.405406e+09,1.893936e+08,5.759569e+09,2.988143e+09,2.270534e+09
max,9.010975e+08,7.014040e+11,1.155640e+11,1.137620e+12,5.654820e+11,6.256140e+14,5.948910e+11,6.620412e+10,1.075490e+12,5.965040e+11,4.789860e+11


In [9]:
# Quitar datos atipicos

q_low = df_prep["INGRESOS OPERACIONALES\n2018*"].quantile(0.01)
q_hi  = df_prep["INGRESOS OPERACIONALES\n2018*"].quantile(0.99)
df_prep = df_prep[(df_prep["INGRESOS OPERACIONALES\n2018*"] < q_hi) & (df_prep["INGRESOS OPERACIONALES\n2018*"] > q_low)]

q_low = df_prep["GANANCIA (PERDIDA) 2018"].quantile(0.01)
q_hi  = df_prep["GANANCIA (PERDIDA) 2018"].quantile(0.99)
df_prep = df_prep[(df_prep["GANANCIA (PERDIDA) 2018"] < q_hi) & (df_prep["GANANCIA (PERDIDA) 2018"] > q_low)]

q_low = df_prep["TOTAL ACTIVOS 2018"].quantile(0.01)
q_hi  = df_prep["TOTAL ACTIVOS 2018"].quantile(0.99)
df_prep = df_prep[(df_prep["TOTAL ACTIVOS 2018"] < q_hi) & (df_prep["TOTAL ACTIVOS 2018"] > q_low)]

q_low = df_prep["TOTAL PASIVOS 2018"].quantile(0.01)
q_hi  = df_prep["TOTAL PASIVOS 2018"].quantile(0.99)
df_prep = df_prep[(df_prep["TOTAL PASIVOS 2018"] < q_hi) & (df_prep["TOTAL PASIVOS 2018"] > q_low)]

q_low = df_prep["TOTAL PATRIMONIO 2018"].quantile(0.01)
q_hi  = df_prep["TOTAL PATRIMONIO 2018"].quantile(0.99)
df_prep = df_prep[(df_prep["TOTAL PATRIMONIO 2018"] < q_hi) & (df_prep["TOTAL PATRIMONIO 2018"] > q_low)]

q_low = df_prep["INGRESOS OPERACIONALES\n2017*"].quantile(0.01)
q_hi  = df_prep["INGRESOS OPERACIONALES\n2017*"].quantile(0.99)
df_prep = df_prep[(df_prep["INGRESOS OPERACIONALES\n2017*"] < q_hi) & (df_prep["INGRESOS OPERACIONALES\n2017*"] > q_low)]

q_low = df_prep["GANANCIA (PERDIDA) 2017"].quantile(0.01)
q_hi  = df_prep["GANANCIA (PERDIDA) 2017"].quantile(0.99)
df_prep = df_prep[(df_prep["GANANCIA (PERDIDA) 2017"] < q_hi) & (df_prep["GANANCIA (PERDIDA) 2017"] > q_low)]

q_low = df_prep["TOTAL ACTIVOS 2017"].quantile(0.01)
q_hi  = df_prep["TOTAL ACTIVOS 2017"].quantile(0.99)
df_prep = df_prep[(df_prep["TOTAL ACTIVOS 2017"] < q_hi) & (df_prep["TOTAL ACTIVOS 2017"] > q_low)]

q_low = df_prep["TOTAL PASIVOS 2017"].quantile(0.01)
q_hi  = df_prep["TOTAL PASIVOS 2017"].quantile(0.99)
df_prep = df_prep[(df_prep["TOTAL PASIVOS 2017"] < q_hi) & (df_prep["TOTAL PASIVOS 2017"] > q_low)]

q_low = df_prep["TOTAL PATRIMONIO 2017"].quantile(0.01)
q_hi  = df_prep["TOTAL PATRIMONIO 2017"].quantile(0.99)
df_prep = df_prep[(df_prep["TOTAL PATRIMONIO 2017"] < q_hi) & (df_prep["TOTAL PATRIMONIO 2017"] > q_low)]

In [10]:
# Obtener cambios del 2017 al 2018

df_prep['CAMBIO EN INGRESOS OPERACIONALES 2017-2018'] = df_prep['INGRESOS OPERACIONALES\n2018*'] - df_prep['INGRESOS OPERACIONALES\n2017*']
df_prep['CAMBIO EN GANANCIA (PERDIDA) 2017-2018'] = df_prep['GANANCIA (PERDIDA) 2018'] - df_prep['GANANCIA (PERDIDA) 2017']
df_prep['CAMBIO TOTAL ACTIVOS 2017-2018'] = df_prep['TOTAL ACTIVOS 2018'] - df_prep['TOTAL ACTIVOS 2017']
df_prep['CAMBIO TOTAL PASIVOS 2017-2018'] = df_prep['TOTAL PASIVOS 2018'] - df_prep['TOTAL PASIVOS 2017']
df_prep['CAMBIO TOTAL PATRIMONIO 2017-2018'] = df_prep['TOTAL PATRIMONIO 2018'] - df_prep['TOTAL PATRIMONIO 2017']

# Modelamiento

# Validación

## Validación cuantitativa

## Validación cualitativa